### 새로나온 책(1-10page)
- 제목
- 저자
- 역자
- 출간일
- 페이지
- 정가
- 판가
- 마일리지

In [2]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
site = 'https://www.hanbit.co.kr'         
url = f'{site}/store/books/new_book_list.html'
res = requests.get(url)

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [6]:
li = lis[1]
href = li.select_one('.book_tit').find('a')['href']
href

'/store/books/look.php?p_code=B6972407557'

In [7]:
title = li.select_one('.book_tit').find('a').get_text()
title

'다시, 배우다'

In [8]:
author = li.select_one('.book_writer').get_text()
author

'폴 김 '

In [9]:
sub_url = site + href
sub_res = requests.get(sub_url)
# sub_html = sub_res.text
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [10]:
sub_lis1 = sub_soup.select('ul.info_list > li')
#sub_lis1

In [11]:
#title = sub_soup.select_one('.store_product_info_box').find('h3').get_text()
#title

In [12]:
#author = sub_lis[0].get_text().split(':')[1].strip()
#author

In [13]:
translator = sub_lis1[1].get_text().split(':')[1:][0].strip()
translator

'2021-12-10'

In [14]:
date = sub_lis1[-4].get_text().split(':')[1].strip()
date

'2021-12-10'

In [15]:
page = sub_lis1[-3].get_text().split(' ')[2]
page

'304'

In [16]:
sub_lis2 = sub_soup.select('div.store_payment_area > fieldset')[0]
#sub_lis2

In [17]:
price = sub_lis2.select('span.pbr > del')[0].get_text()
price

'16,000원'

In [18]:
#price = sub_lis2.select('span.pbr')[0].get_text()
#price
#container > div.store_view_wrap > div.store_payment_area > fieldset > label.payment_box.curr > p:nth-child(1) > span.pbr > del

In [19]:
sel_price = sub_lis2.select('span.pbr > strong')[0].get_text().strip()
sel_price

'14,400'

In [20]:
#sel_price = sub_lis2.select('span.pbr')[1].get_text().split('(')[0]
#sel_price

In [21]:
point = sub_lis2.select('span.pbr')[2].get_text().split(' ')[0]
point

'800점'

In [22]:
def convert_unit(s):
    s = s.replace('원','').replace('점','').replace(',','')
    return f'{int(s):,d}'

In [23]:
lines = []
for li in lis:
    href = li.select_one('.book_tit').find('a')['href']
    title = li.select_one('.book_tit').find('a').get_text()
    author = li.select_one('.book_writer').get_text()

    sub_url = site + href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis1 = sub_soup.select('ul.info_list > li')
    date = sub_lis1[-4].get_text().split(':')[1].strip()
    page = sub_lis1[-3].get_text().split(' ')[2]
    sub_lis2 = sub_soup.select('div.store_payment_area > fieldset')[0]
    price = sub_lis2.select('span.pbr > del')[0].get_text()
    sel_price = sub_lis2.select('span.pbr > strong')[0].get_text().strip()
    point = sub_lis2.select('span.pbr')[2].get_text().split(' ')[0]
    lines.append([title,author,date,convert_unit(page),convert_unit(price),convert_unit(sel_price),convert_unit(point)])

In [24]:
df = pd.DataFrame(lines, columns=['책제목','저자','출간일','쪽수','정가','판매가','마일리지'])
df.head()

,책제목,저자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,2021-12-14,416,"19,800","17,820",990
1,"다시, 배우다",폴 김,2021-12-10,304,"16,000","14,400",800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,2021-12-10,376,"22,000","19,800","1,100"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",2021-12-01,384,"18,000","16,200",900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale",2021-11-30,600,"36,000","36,000","1,080"


In [25]:
df.tail()

,책제목,저자,출간일,쪽수,정가,판매가,마일리지
15,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),2021-11-05,240,"16,800","15,120",840
16,구글 BERT의 정석,수다르산 라비찬디란,2021-11-03,368,"34,000","30,600","1,700"
17,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,2021-11-02,96,"8,000","7,200",400
18,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,2021-11-02,96,"8,000","7,200",400
19,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,2021-11-02,96,"8,000","7,200",400


In [26]:
lines = []
for page in range(1,11):
    url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(page)     # '&brand=&cate1=&cate2=&searchKey=&keyWord=' 뒷부분은 parameter가 지정되지 않았기 때문에 없어도 상관없음 / 'store/books/new_book_list.html?page=' 이 부분이 path인데 크롤링할 때 url에서 신경써야 하는 부분
    soup = BeautifulSoup(res.text, 'html.parser')
    li = lis[0]
 
    lines = []
    for li in lis:
        href = li.select_one('.book_tit').find('a')['href']
        title = li.select_one('.book_tit').find('a').get_text()
        author = li.select_one('.book_writer').get_text()

        sub_url = site + href
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis1 = sub_soup.select('ul.info_list > li')
        date = sub_lis1[-4].get_text().split(':')[1].strip()
        page = sub_lis1[-3].get_text().split(' ')[2]
        sub_lis2 = sub_soup.select('div.store_payment_area > fieldset')[0]
        price = sub_lis2.select('span.pbr > del')[0].get_text()
        sel_price = sub_lis2.select('span.pbr > strong')[0].get_text().strip()
        point = sub_lis2.select('span.pbr')[2].get_text().split(' ')[0]
        lines.append([title,author,date,convert_unit(page),convert_unit(price),convert_unit(sel_price),convert_unit(point)])

In [29]:
df = pd.DataFrame(lines, columns=['책제목','저자','출간일','쪽수','정가','판매가','마일리지'])
df.head()

,책제목,저자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,2021-12-14,416,"19,800","17,820",990
1,"다시, 배우다",폴 김,2021-12-10,304,"16,000","14,400",800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,2021-12-10,376,"22,000","19,800","1,100"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",2021-12-01,384,"18,000","16,200",900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale",2021-11-30,600,"36,000","36,000","1,080"


In [30]:
df.tail()

,책제목,저자,출간일,쪽수,정가,판매가,마일리지
15,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),2021-11-05,240,"16,800","15,120",840
16,구글 BERT의 정석,수다르산 라비찬디란,2021-11-03,368,"34,000","30,600","1,700"
17,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,2021-11-02,96,"8,000","7,200",400
18,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,2021-11-02,96,"8,000","7,200",400
19,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,2021-11-02,96,"8,000","7,200",400


- 시리즈가 여러개 있을 경우 각 시리즈의 갯수가 동일해야 함 만약 값이 없다면 '-'이나 'NA' 혹은 'NaN'으로 채워야 함
- 코딩을 할 때는 완료한 후 refactoring하는 과정이 필요